In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
data2001 = pd.read_csv('./pems12/d12_text_station_5min_2018_02_28.txt.gz',header=None, usecols=range(12))
# Assign column names based on the provided headers
column_names = [
    "Timestamp", "Station", "District", "Freeway #", 
    "Direction of Travel", "Lane Type", "Station Length", 
    "Samples", "% Observed", "Total Flow", "Avg Occupancy", "Avg Speed"
]

# Assign column names to the dataframe
data2001.columns = column_names
data2001

,Timestamp,Station,District,Freeway #,Direction of Travel,Lane Type,Station Length,Samples,% Observed,Total Flow,Avg Occupancy,Avg Speed
0,02/28/2018 00:00:00,1201044,12,133,S,OR,NaN,0,0,NaN,NaN,NaN
1,02/28/2018 00:00:00,1201052,12,133,S,FR,NaN,0,0,NaN,NaN,NaN
2,02/28/2018 00:00:00,1201054,12,133,S,ML,1.285,0,0,12.0,0.0046,68.7
3,02/28/2018 00:00:00,1201058,12,133,N,OR,NaN,0,0,NaN,NaN,NaN
4,02/28/2018 00:00:00,1201064,12,133,N,FR,NaN,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
732955,02/28/2018 23:55:00,1221220,12,57,N,HV,0.400,9,100,6.0,0.0033,65.1
732956,02/28/2018 23:55:00,1221232,12,5,N,ML,0.575,0,0,191.0,0.0196,68.4
732957,02/28/2018 23:55:00,1221233,12,5,S,ML,0.575,0,0,191.0,0.0196,68.4
732958,02/28/2018 23:55:00,1221234,12,5,N,HV,0.575,0,0,48.0,0.0196,68.4


In [3]:
comm = np.load('pems12_comm.npy')
comm

array([1201052, 1201054, 1201064, ..., 1212690, 1212691, 1212692],
      dtype=int64)

In [4]:
import pandas as pd
import calendar
from datetime import date, timedelta

def read_and_process_data(file_path):
    try:
        # Read the CSV file, specifying the usecols parameter to only load the columns of interest
        data = pd.read_csv(
            file_path, 
            header=None, 
            usecols=[0, 1, 9],  # Column indexes for Timestamp, Station, and Total Flow
            names=['Timestamp', 'Station', 'Avg Flow'],  # Assigning column names
            compression='gzip'
        )

        # Convert the Timestamp column to datetime format and set it as the index
        data['Timestamp'] = pd.to_datetime(data['Timestamp'])
        data.set_index('Timestamp', inplace=True)

        # Pivot the table to get Stations as columns and Total Flow as cell values
        pivot_data = data.pivot(columns='Station', values='Avg Flow')

        return pivot_data

    except EOFError:
        print(f"Error processing file: {file_path}")
        return None

# Read the data for February 28 to get the column names (Station IDs)
# feb_28_data = read_and_process_data('./pems03/d03_text_station_5min_2018_02_28.txt.gz')
# feb_28_columns = feb_28_data.columns

# Initialize a list to hold the data for all days
all_data = []

start_date = date(2002, 1, 1)  # 开始日期
# end_date = date(2012, 10, 2)   # 结束日期
end_date = date(2024, 3, 20)   # 结束日期
current_date = start_date
all_data = []                  # 存储所有数据的列表

while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    file_path = f'./pems12/d12_text_station_5min_{year}_{month:02d}_{day:02d}.txt.gz'
#     if year==2012 and month==10 and day==1:
#         continue
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}, skipping...")
        current_date += timedelta(days=1)
        continue
    
    day_data = read_and_process_data(file_path)
    
    # 使用字典存储新列，以便一次性添加到数据帧
    new_columns = {}
    for col in comm:
        if col not in day_data:
            new_columns[col] = 0  # 假设使用0填充缺失的列

    # 使用 pd.concat 添加所有新列
    if new_columns:
        new_data = pd.DataFrame(new_columns, index=day_data.index)
        day_data = pd.concat([day_data, new_data], axis=1)

    # 将列重新排序以匹配2月28日的顺序
    day_data = day_data[comm]
    
    all_data.append(day_data)
    
    current_date += timedelta(days=1)

# Concatenate all daily data into a single DataFrame
combined_data = pd.concat(all_data)

# Fill any remaining missing values with 0
combined_data.fillna(0, inplace=True)

# The combined_data DataFrame now holds the merged data for January 1 to February 28
# with columns aligned to February 28 and missing values filled with 0
# Note: The code execution is commented out to prevent execution in this environment.
# combined_data.head()
combined_data.to_csv('pems12_all_common_flow.csv')

In [ ]:
combined_data

In [ ]:
import pandas as pd
pems12 = pd.read_csv('pems12_all_common_flow.csv')

# Convert the 'date' column to datetime
pems12['date'] = pd.to_datetime(pems12['date'])

# Set the 'date' column as the index
pems12.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems12_hourly = pems12.resample('H').sum()

# Reset the index if you want the 'date' column back
pems12_hourly.reset_index(inplace=True)
pems12_hourly.to_csv('pems12_h.csv',index=False)


# Convert the 'date' column to datetime
pems12 = pd.read_csv('pems12_all_common_flow.csv')
pems12['date'] = pd.to_datetime(pems12['date'])

# Set the 'date' column as the index
pems12.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems12_daily = pems12.resample('D').sum()

# Reset the index if you want the 'date' column back
pems12_daily.reset_index(inplace=True)
pems12_daily.to_csv('pems12_d.csv',index=False)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_35880\3172734862.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pems12_hourly = pems12.resample('H').sum()
